In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import average_precision_score

import xgboost as xg

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
train = pd.read_csv('../input/santander-customer-transaction-prediction/train.csv')
test  = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')

In [ ]:
train.head(5)

In [ ]:
train.info()

In [ ]:
test.head(5)

In [ ]:
test.info()

In [ ]:
def func(df):
    a = df.isnull().sum()
    b = df.count()
    c = (a/b) * 100
    d = pd.DataFrame(a, columns = ['Missingvalue%'])
    return d['Missingvalue%'].sum()

In [ ]:
print('missing values in train data:', func(train))
print('missing values in test data:', func(test))

#输出判断无缺失值，不用做数据清洗

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
test_df = test
test_id_list = test_df.ID_code
test_df.drop(['ID_code'],axis=1,inplace=True)

In [ ]:
df = train
print(df["target"].value_counts()/df.shape[0])
sns.countplot("target", data = df)

# 结论:数据高度不平衡, target(Y) column value’1‘占比10%

In [ ]:
df.corr()

In [ ]:
y = df["target"]
x = df.drop(columns=["target", "ID_code"]).values

In [ ]:
#model set
x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size = 0.3, random_state = 100)

**XGBoost**

In [ ]:
model = xg.XGBClassifier(objective ='binary:logistic', 
                         colsample_bytree = 0.3, 
                         learning_rate = 0.03,
                         max_depth = 6,
                         alpha = 10,
                         n_estimators = 100,
                         seed=100)

model.fit(x_train,y_train)
model.score(x_test_val,y_test_val)


In [ ]:
#First optimization
model_op1 = xg.XGBClassifier(objective ='binary:logistic', 
                             colsample_bytree = 0.8,
                             colsample_bylevel=0.8,
                             learning_rate = 0.1,
                             max_depth = 5, 
                             alpha = 10, 
                             n_estimators = 100,
                             subsample=0.8,
                             gamma=0,
                             scale_pos_weight=3,
                             nthread=4,
                             seed=100)

model_op1.fit(x_train,y_train)
model_op1.score(x_test_val,y_test_val)

In [ ]:
#sec try  optimization
model_op2 = xg.XGBClassifier(objective ='binary:logistic', 
                             colsample_bytree = 0.8,
                             colsample_bylevel=0.8,
                             learning_rate = 0.03,
                             max_depth = 4, 
                             alpha = 1, 
                             n_estimators = 100,
                             subsample=1,
                             min_child_weight=1,
                             scale_pos_weight=3,
                             nthread=4,
                             seed=100)

model_op2.fit(x_train,y_train)
model_op2.score(x_test_val,y_test_val)

In [ ]:
# Last optimization
classifier = xg.XGBClassifier(n_thread = -1, tree_method='gpu_hist')
param_grid = {
    "n_estimators" : np.arange(100, 500, 50),
    "max_depth" : np.arange(3, 6, 1),
    "colsample_bytree": np.arange(0.5,1, 0.1),
    "criterion": ["gini",'entropy']
}
model3 = RandomizedSearchCV(estimator = classifier,
                          param_distributions = param_grid,
                          n_iter = 10,
                          scoring = "accuracy",
                          verbose = 10,
                          n_jobs = -1,
                          cv = 5)
model3.fit(x_train, y_train)
model3.best_score_

In [ ]:
print(model3.best_estimator_.get_params())

In [ ]:
accuracy_test = accuracy_score(y_test_val, model3.predict(x_test_val))
print("Accuracy on test %.2f%%" % (accuracy_test * 100))

In [ ]:
# submitting output
output_submission = pd.DataFrame(zip(test_id_list,model3.predict_proba(test_df)[:,1]), columns = ['ID_code','target'])
output_submission.to_csv('/kaggle/working/output_submission.csv',index=False)